# TCP/IP
In this exercise set, a virtual instrument is made available via the TCP/IP protocol.
In order to communicate with this instrument it is not necessary to know every detail of this protocol as there is a python package available to handle the details for you. To get an idea of the TCP/IP protocol, you can take a look at this page:
https://en.wikipedia.org/wiki/Internet_protocol_suite

The important thing for this exercise set is to know how to communicate via TCP/IP using a python library.
We will be using the socket library, see:
https://docs.python.org/3/library/socket.html

This library can be imported into python with

In [ ]:
import socket

To communicate with a target machine, you will need to know its IP address and the port number.
We have set up your python containers to run the measurement server as well, hence the adress will be

'localhost' 

or 

'127.0.0.1'

The port number for the multimeter is 

5000

You can instantiate a socket object with

In [ ]:
socket.socket(socket.AF_INET, socket.SOCK_STREAM)

As a first exercise, you will need to send a single command to the multimeter server. You can look up the appropriate commands on the website above, or by using google.
Take care that you can only send and receive 'bytes' objects. You can from and to strings by using '.encode()' and '.decode()' resepctively.

For your convenience, the commands that you can send are given below:

SENSe:DATA? - read the voltage over the resistor

SOUR:CURR:LEV? - read the current being send through the resistor

SOUR:CURR:LEV # - set the current through the resistor

## exercise 1
1. Open a connection to the instrument
2. Send a command to the instrument to read the voltage
3. Print the result (do not worry about the result yet)
4. Close the connection to the instrument

In [ ]:
import socket
address = '127.0.0.1'
port = 5000
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((address, port))
sock.send(b'SENS:DATA?')
print(sock.recv(1024))
sock.close()

# Making a driver
In order to making something useful, you will need to make functions to perform specific tasks on your multimeter.
Take a collegue in mind and realise that this person does not want to know about TCP/IP, or about the specific commands that he/she has to send to the device. This person will just want to call some specific python functions.

It is your task, as writer of the driver, to make this possible by writing these functions and making sure that they will be as stable as possible

## exercise 2
1. Write separate functions to set the current and read the voltage
2. Change the read function to return a number instead of a string
3. Measure the value of your resistor

In [ ]:
import socket
address = '127.0.0.1'
port = 5000
def ask(cmd):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((address, port))
    sock.send(cmd)
    r = sock.recv(1024)
    sock.close()
    return r

def get_voltage():
    r = ask(b'SENS:DATA?').decode()
    v = float(r.split(' ')[1][:-1])
    return v

def set_current(i):
    return ask(b''+str(i).encode())

current = 0.5
print(set_current(current))
volt = get_voltage()
print(volt / current)

# Making everything more stable
As you may have noticed, the instrument can return things that you did not expect. In this exercise you need the check the output of the multimeter, to make sure that you can take many consecutive measurement points without any gaps in the data.

Note that you must make your functions stable against faults in the measurement device. Do not try to solve this problem in your data acquisition loop.

## exercise 3
1. Add error handling to your functions
2. Average the results to reduce the noise
3. Make a graph of voltage vs. current

In [ ]:
import socket
import time
import numpy as np
import matplotlib.pyplot as plt
address = '127.0.0.1'
port = 5000
def ask(cmd):
    r = b''
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        
        sock.settimeout(1)
        sock.connect((address, port))
        sock.send(cmd)
        r = sock.recv(1024)
    except:
        print('Connection problems')
    finally:
        sock.close()
        
    return r

def get_voltage():
    for i in range(5):
        r = ask(b'SENS:DATA?').decode()
        if r.startswith('SENS'):
            break
        time.sleep(i)
    if not r.startswith('SENS'):
        print('Getting voltage failed')
        return 0
    v = 0
    try:
        v = float(r.split(' ')[1][:-1])
    except:
        print('Decoding voltage failed')
        return 0
    return v

def set_current(I):
    for i in range(5):
        r = ask(('SOUR:CURR:LEV'+str(I)).encode()).decode()
        if r.startswith('SOUR'):
            break
        time.sleep(i)
    if not r.startswith('SOUR'):
        print('Setting current failed')
        return 0
    return r

vs = []
avg = 10
Is = np.linspace(-0.5,0.5,100)
for I in Is:
    set_current(I)
    vss = []
    for i in range(avg):
        vss.append(get_voltage())
    vs.append(np.mean(vss))

plt.plot(Is, vs)
plt.show()

# Making your driver into a driver
A driver is often a bit more than a collection of functions.
In most cases, instruments have a state with them and this is one of the cases where it is useful to use an object to represent your instrument.

By writing a class for your instrument, you can also have multiple instruments at the same time. By having all instruments in different (instances of) classes, it becomes easier to address every instrument individually.

In the next exercise you will connect to two different instruments, but you will only have to write a single class to do it. By making two instances of this class, you will be able to address the two multimeters in turn.

## exercise 4
(more advanced)

1. Turn all this code into a class.
2. create two instances of this class, and try to communicate to both your own multimeter, and the multimeter of your neighbour

   hint: You will need to know the ip address of your neighbour. You can find this by typing "hostname -I" in your terminal

In [ ]:
class multimeter:
    def __init__(self, adress, port):
        self.address = address
        self.port = port
    
    def _ask(self, cmd):
        r = b''
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:

            sock.settimeout(1)
            sock.connect((self.address, self.port))
            sock.send(cmd)
            r = sock.recv(1024)
        except:
            print('Connection problems')
        finally:
            sock.close()

        return r
    
    def get_voltage(self):
        for i in range(5):
            r = self._ask(b'SENS:DATA?').decode()
            if r.startswith('SENS'):
                break
            time.sleep(i)
        if not r.startswith('SENS'):
            print('Getting voltage failed')
            return 0
        v = 0
        try:
            v = float(r.split(' ')[1][:-1])
        except:
            print('Decoding voltage failed')
            return 0
        return v

    def set_current(self, I):
        for i in range(5):
            r = self._ask(('SOUR:CURR:LEV'+str(I)).encode()).decode()
            if r.startswith('SOUR'):
                break
            time.sleep(i)
        if not r.startswith('SOUR'):
            print('Setting current failed')
            return 0
        return r

In [ ]:
m = multimeter('127.0.0.1', 5000)
m.set_current(0.25)
print(m.get_voltage())

# A more complicated instrument
We also have made a classical simulation of a quantum computer available to you. If you find this interesting you might find it useful to try to communicate with this instrument.
the address is again your local host (127.0.0.1) and the port number is 5001

The quantum computer will have 4 qubits for you to play with.
If you want the execute Shors algorithm to factor 15, you need 7 qubits, we can make those available to you if you want.

The list of commands that you can execute on the quantum computer is

OP:[X90|Y90|X45|y45]:Q[n] (Rotate around X or Y, 90 or 45 degrees)

OP:CNOT:Qn:Qm (Perform a CNOT gate)

OP:ZERO (reset the quantum computer)

OP:MEASURE (obtain a classical readout of the quantum computer, this will destroy the state!)

Here, m,n are the indices of your qubits (starting from 0)


## exercise 5
(more advanced)

Use your knownledge aquired today to control a quantum computer. This exercise can be combined with the QASM advanced project. Try to read in a QASM file and execute the corresponding algorithm on this quantum computer.